# 5일차: 위젯

## 위젯 기본

### pack

In [1]:
from tkinter import Tk, Button

# Creates window
window = Tk()

button = Button(window, text='Ok')
button.pack()

# Blocks until window closes
window.mainloop()

### grid

In [4]:
from tkinter import Tk, Button, Label

# Creates window
window = Tk()

button = Button(window, text='Ok')
button.grid(row=0, columnspan=2)

text = Label(window, text='Hello, world!', background='#ffffff', foreground='#3378f0')
text.grid(row=1, columnspan=2)

Label(window, text='hOI!!!!', foreground='#ff3200').grid(row=2, column=0)
Label(window, text='lhwdev', foreground='#003300').grid(row=2, column=1)

# Blocks until window closes
window.mainloop()

### layout test

In [5]:
from tkinter import Tk, Button, Label

# Creates window
window = Tk()

Label(window, text='a', foreground='#ff3200').grid(row=0, column=0)
Label(window, text='b', foreground='#22ff67').grid(row=1, column=1)
Label(window, text='c', foreground='#22ff80').grid(row=2, column=2)

# Blocks until window closes
window.mainloop()

In [6]:
from tkinter import Tk, Button, Label

# Creates window
window = Tk()

Label(window, text='a', foreground='#ff3200').place(x=0, y=0)
Label(window, text='b', foreground='#22ff67').place(x=40, y=40)
Label(window, text='c', foreground='#22ff80').place(x=80, y=80)

# Blocks until window closes
window.mainloop()

## Inch <-> cm calculator

In [14]:
from tkinter import Tk, Frame, Label, StringVar, Entry, Button

# Creates window
window = Tk()

# Values
inch = StringVar()
cm = StringVar()
modification = ('none', '0')

# Inch row
inchRow = Frame(window)

Label(inchRow, text='Inch').pack(side='left')

def inch2cm(*args):
  global modification
  if modification[0] == 'cm' and modification[1] == inch.get():
    return

  try:
    newCm = str(2.54 * float(inch.get()))
    modification = ('inch', newCm)
    cm.set(newCm)
  except ValueError:
    pass

inch.trace_add('write', inch2cm)
inchEntry = Entry(inchRow, textvariable=inch)
inchEntry.pack(side='left')

inchRow.pack()

# cm row
cmRow = Frame(window)

Label(cmRow, text='cm').pack(side='left')


def cm2inch(*args):
  global modification
  if modification[0] == 'inch' and modification[1] == cm.get():
    return

  try:
    newInch = str(float(cm.get()) / 2.54)
    modification = ('cm', newInch)
    inch.set(newInch)
  except ValueError:
    pass

cm.trace_add('write', cm2inch)
cmEntry = Entry(cmRow, textvariable=cm)
cmEntry.pack(side='left')

cmRow.pack()

# Button row
# buttonRow = Frame(window)
#
# inch2cm = Button(buttonRow, text='Inch -> cm', command=)
# inch2cm.pack(side='left')
#
# cm2inch = Button(buttonRow, text='cm -> Inch')
# cm2inch.pack(side='left')
#
# buttonRow.pack()


# Blocks until window closes
window.mainloop()

## 계산기
- 숫자 입력 버튼
- 연산 지원

In [110]:
from abc import *
from typing import *

# 연산자 토큰들의 종류. ','는 함수 호출 시 인수들을 구분하는데 쓰임
tOperators = ['+', '-', '*', '/', '^', '%', ',']
# 연산자 우선순위: 높은 연산자가 우선적으로 계산됨
# 우선순위가 같다면 앞부터 순서대로 계산됨, 괄호의 경우 항상 최우선적으로 실행
# 따라서 1 + 2 * 3 ^ (4 - 5) - 6의 경우 [{1 + (2 * [3 ^ (4 - 5)])} - 6]처럼 묶임
tOperatorPrecedences = {
  '^': 15,
  '*': 10,
  '/': 10,
  '%': 10,
  '+': 3,
  '-': 3,
  
  # stubs: 모든 연산자는 일단 우선순위를 지정해야 하기에... 임시로 둠
  ',': -1
}
# 이항 연산자: a + b처럼 이항연산을 하는 경우
binaryOperators = ['+', '-', '*', '/', '%', '^']
# 단항 연산자: +a, -a처럼 머리에 붙어서 한 숫자와만 관여하는 경우
unaryOperators = ['+', '-']
# 가능한 숫자의 종류: 123.45 같은거
tDigits = '0123456789e.'
# 가능한 글자의 종류: 파이썬 내장함수인 isalpha에 의존하기로 함 (구현하기 귀찮아요 ㅠㅠ)

# '토큰': 평면적인 연산자, 숫자, 괄호 등을 추상화함
# 클래스는 예전에 다른 프로그래밍 수업에서 배운거라... 대충 데이터들을 묶어놓은 것이라 생각하면 됩니다
class Token:
  # 종류: 'number'(숫자), 'operator'(연산자), 'group/group_close'(괄호 열기/닫기), 'text'(글자: 함수의 이름 등: 'cos', 'tan', 'f', 'x' 등이 가능함)
  kind: str
  # 해당하는 코드 부분: '123', 'cos', '*', '(', ')' 등
  code: str
  # 연산자라면 우선순위: tOperatorPrecedences에서 불러온 값
  precedence: int

  def __init__(self, kind: str, code: str, precedence: int):
    self.kind = kind
    self.code = code
    self.precedence = precedence

  def __repr__(self):
          return f'{self.code}'

# 그냥 글자 ('1 + 3 * 4' 같은)을 받아서 Token들로 바꿔주는 (숫자 1, 연산자 +, 숫자 3, 연산자 *, 숫자 4) 공장?
class Tokenizer:
  index: int = 0
  code: str

  def __init__(self, code: str):
    self.code = code

  # 끝까지 다 봤는지 여부
  def eof(self):
    return self.index == len(self.code)
  
  # 다음 '토큰'을 불러오는 함수
  def advance(self):
    while True: # 띄어쓰기는 무시하기 때문에 띄어쓰기를 제외한 다음 토큰을 찾아서 반환함
      start = self.index
      c = self.code[start]
      self.index += 1
      
      # match는 if .. else if .. else가 귀찮을 때 쓸 수 있음
      # 아래 코드는 if c in tOperators: .. else if c in tDigits: .. else if c == '(': .. 와 같음
      # 지금 텍스트가
      match c:
        case '(': # 괄호 열기
          return Token(kind='group', code=c, precedence=1000)
        case ')': # 괄호 닫기
          return Token(kind='group_close', code=c, precedence=1000)
        case _ if c in tOperators: # 연산자
          return Token(kind='operator', code=c, precedence=tOperatorPrecedences[c])
        case _ if c.isdigit(): # 숫자
          # Parse number
          while (not self.eof()) and self.code[self.index] in tDigits:
            self.index += 1
          return Token(kind='number', code=self.code[start:self.index], precedence=0)
        case _ if c.isalpha(): # 글자
          while (not self.eof()) and self.code[self.index].isalpha():
            self.index += 1
          return Token(kind='text', code=self.code[start:self.index], precedence=0)
        case _:
          if c.isspace(): # 띄어쓰기는 무시
            continue
          raise Exception('Malformed expression')

def debugTokens(tokens: List[Token]):
  return {"".join([t.code for t in tokens])}

# 토큰들의 목록. tokens[a:b]는 새로운 리스트를 만들기 때문에 메모리 사용량이 많아질 수 있음.
# 대신 TokenReference.ofEnd(tokens, a, b)라는걸 만듦
class TokenReference:
  # tokens
  allTokens: List[Token]
  # a
  start: int
  # b
  end: int

  def __init__(self, allTokens: List[Token], start: int, end: int):
    self.allTokens = allTokens
    self.start = start
    self.end = end

  @staticmethod
  def ofEnd(allTokens: List[Token], start: int, end: int):
    return TokenReference(allTokens, start, end)

  @staticmethod
  def ofCount(allTokens: List[Token], start: int, count: int):
    return TokenReference(allTokens, start, start + count)
  
  # operator라고 해서 이것[index] 같은 것을 할 수 있게 함
  def __getitem__(self, index: int):
    return self.allTokens[self.start + index]
  
  def __len__(self):
    return self.end - self.start

  def __repr__(self):
    return f'TokenReference({self.allTokens[self.start:self.end]})'


# '노드'는 수식의 구조적인 것을 나타냄.
# 보통 이런 '노드'를 AST(Abstract Syntax Tree)라고 하는데, 구문(수학의 문법; 곱하기 나누기 같은 스타일이라 해야하나)을 추상적인 트리 형식으로 나타낸 것.
# 예를 들어 1 + 2 * 3이라는 수식은 1 더하기 (2 곱하기 3)이라고 나타낼 수 있고, 이걸 '노드'를 통해 나타내면 대략적으로
# Add(1, Multiply(2, 3))처럼 나타낼 수 있음. (실제로 이 코드에서는 조금 다름)
class Node:
  parent: Optional['Node']
  tokens: TokenReference
  kind: str

  # 이 '노드'를 구성하는 토큰들이 원래 뭐였는지 포함하면 개발이 편할 것 같아서 tokens를 받게 해놨음
  def __init__(self, tokens):
    self.tokens = tokens

# 그룹: 괄호로 묶인 부분
class Group(Node):
  node: Node

  # 클래스를 '만들 때' 호출됨.
  # 따라서 Group(tokens, node)라고 쓸 수 있음.
  def __init__(self, tokens, node):
    super().__init__(tokens)
    self.node = node
    self.kind = 'group'
  
  # 참고: __repr__은 그 클래스의 값을 문자열로 바꿀 때 기본적으로 호출됨.
  # print(node)를 할 때 겁나 괴랄하게 출력된다면 버그 고치기 힘들 것 같아서 만들음.
  def __repr__(self):
    return f'{self.node}'
  
  # 원본 수식으로 되돌리는 함수
  def code(self):
    return node.code()

# 이항 연산자: left + operator같은 것들
class BinaryOperator(Node):
  left: Node
  right: Node
  operator: str

  def __init__(self, tokens, left, right, operator):
    super().__init__(tokens)
    self.left = left
    self.right = right
    self.operator = operator
    self.kind = 'binary_operator'

  def __repr__(self) -> str:
    return f'BinaryOperator({self.left}, "{self.operator}", {self.right})'
  
  def code(self): # left * right
    return f'({self.left} {self.operator} {self.right})'

# 단항 연산자: -value 같은 것
class UnaryOperator(Node):
  value: Node
  operator: str

  def __init__(self, tokens, value, operator):
    super().__init__(tokens)
    self.value = value
    self.operator = operator
    self.kind = 'unary_operator'
  
  def __repr__(self) -> str:
    return f'UnaryOperator("{self.operator}", {self.value})'
  
  def code(self): # -123
    return f'({self.operator}{self.value})'

# 숫자 그 자체
class Number(Node):
  number: float

  def __init__(self, tokens, number):
    super().__init__(tokens)
    self.number = number
    self.kind = 'number'
  
  def __repr__(self) -> str:
    return f'{self.number}'
  
  def code(self): # 123
    return f'{self.number}'
    
# 문자: 'x'처럼 변수나 상수가 되는 것들
class Symbol(Node):
  symbol: str

  def __init__(self, tokens, symbol):
    super().__init__(tokens)
    self.symbol = symbol
    self.kind = 'symbol'
  
  def __repr__(self) -> str:
    return f'Symbol("{self.symbol}")'
  
  def code(self): # x
    return f'{this.symbol}'

# 그냥 말그대로 함수.
# 내장함수는 이거가 기본으로 제공됨.
class Function:
  calculate: Callable[[List[float]], float]

  def __init__(self, calculate):
    self.calculate = calculate

# 함숫값: 함수를 호출한 값을 나타냄
# 예시: f(x), sin(2 * PI) 등
class FunctionValue(Node):
  name: str # 함수 이름
  targetFunction: Function # 함수를 호출할 때 호출하는 곳. sin, cos, tan같은 내장함수의 경우 자동으로 지정됨.
  args: List[Node] # 인수들. 리스트로 둬서 다변수함수도 구현할 수 있음

  def __init__(self, tokens, name, args):
    super().__init__(tokens)
    self.name = name
    self.args = args
    self.kind = 'function_value'
  
  def __repr__(self) -> str:
    return f'FunctionValue("{self.name}", args={self.args})'
  
  def code(self): # function_name(arg1, arg2, ...)
    return f'{self.name}({", ".join([str(arg) for arg in self.args])})'


간단한 수식 계산기: **수식 파싱 기능**
- 연산자 우선순위 구현: `*`, `/` 연산자가 `+`, `-` 연산자보다 먼저 계산됨 (괄호는 말할 것도 없고)
- AST를 출력하는 파서 부분, ~~컴파일러~~, AST를 가지고 실행하는 인터프리터 구현
- top-down 파서임

In [102]:
'''
간단한 수식 계산기 파서 (파서 = 토큰들을 '노드'로 바꾸는 것)
.. '간단한'?

고려해야 할 점:
- 연산자 우선순위 구현: `*`, `/` 연산자가 `+`, `-` 연산자보다 먼저 계산됨 (괄호는 말할 것도 없고)
  이게 골때림. 개발하는데 힘들었어요 ㅠㅠ

따라서 이 함수는
1.  괄호를 인식함
    괄호를 본다면 급발작 스위치가 발동해서 괄호 부분을 먼저 파싱함
2.  사용된 연산자들을 우선순위별로 분류함
3.  우선순위가 높은 연산자부터 계산함

대략적인 작동방법을 보자면,
코드:          1  +  2  *  (  3  +  4  *  5  )  +  6
(괄호 먼저 계산)                                                    
연산자 우선순위:                   2     1             <- 리스트에 현재 담겨있는 내용
- 우선순위 = 1:                      A  A  A          (A = 4 * 5)
- 우선순위 = 2:                B  B  B  B  B          (B = 3 + A)
(괄호 밖)
연산자 우선순위:    2     1  B  B  B  B  B  B  B  2   
- 우선순위 = 1:    2  C  C  C  C  C  C  C  C  C  2    (C = 2 * B)
- 우선순위 = 2:    D  D  D  D  D  D  D  D  D  D  2    (D = 1 + C) <- 1페이즈
- 우선순위 = 2:    E  E  E  E  E  E  E  E  E  E  E  E (E = D + 6) <- 2페이즈
완성된 AST: E = [{1 + (2 * {3 + (4 * 5)})} + 6]


tokens: 전체 토큰들
start: .. 중에서 시작 위치,
end: 끝 위치
'''
def parseToAst(tokens: List[Token], nodes, start: int, end: int) -> Node:
  if start == end:
    raise Exception(f'Unknown index: {start} to {end}')
  
  # 범위 내에서 첫번째 토큰이
  firstToken = tokens[start]

  def parseText(index):
    s = index
    text = tokens[index]
      # 뒤에 괄호가 오면 함수 호출
    if index + 1 < end and tokens[index + 1].kind == 'group':
      # 함수 호출
      
      index += 2 # 괄호 안부터 시작

      # 인수들을 각자 파싱함
      args = []
      while index < end:
        arg = parseToAst(tokens, nodes, index, end)
        args.append(arg)
        index += len(arg.tokens)

        if tokens[index].code == ',': # ,로 끝남
          index += 1
          continue
        else: # )로 끝남
          index += 1
          break
      
      return FunctionValue(TokenReference.ofEnd(allTokens=tokens, start=s, end=index), name=text.code, args=args)
    
    else: # 뒤에 괄호가 오지 않음. 단순한 변수/상수
      return Symbol(TokenReference.ofCount(allTokens=tokens, start=s, count=1), symbol=text.code)


  
  match firstToken.kind:
    case 'group': # 괄호 열기
      node = parseToAst(tokens, nodes, start + 1, end)
      return Group(TokenReference.ofCount(allTokens=tokens, start=start, count=len(node.tokens) + 2), node)
    
    case 'operator': # 연산자: 처음부터 이게 오면 단항 연산자임
      value = parseToAst(tokens, nodes, start + 1, end)
      return UnaryOperator(TokenReference.ofCount(allTokens=tokens, start=start, count=len(value.tokens) + 1), value)

    case 'text': # 글자
      return parseText(start)
    
    case 'number': # 숫자. 숫자가 처음에 오면 그냥 숫자가 끝일 수도 있지만 1 * 3 같은 식에서도 1이 맨 앞에 옴
      # 따라서 뒤에 뭐가 따라오는지 확인해야 함

      # 1. Check operator precedences
      #   sort operators by precedence: create list -> put operator precedences in it
      # 2. Parse expression
      #   execute the operator with high precedence first, top-down
      # 3. Return result

      # 1. Check operator precedences
      # 노드/우선순위들을 임시로 담아두는 리스트
      # nodes = [None] * nodesCount # elements: int = precedence, Node = parsed node
      # 우선순위들을 모아놓은 집합
      allPrecedences = set()

      # 범위 안의 토큰들을 조사함
      index = start
      while index < end:
        current = tokens[index]

        # In case of group
        match current.kind:
          case 'group': # 말했다시피 괄호를 보면 바로 발작함
            # 괄호 안의 내용물을 파싱함
            groupValue = parseToAst(tokens, nodes, index, end)

            # 위 작동방법에서 그룹 전체를 'B'로 채우는 것과 같은 과정
            for ii in range(index, index + len(groupValue.tokens)):
              nodes[ii] = groupValue
            index += len(groupValue.tokens) # 그룹 + 그 안의 총 토큰 개수만큼 이동
          
          case 'group_close': # 괄호닫기. 바로 위에서 그룹을 파싱할 때 이게 호출될 때까지 반복함
            break # while문에서 빠져나오기

          case 'text':
            node = parseText(index)
            
            # 위 작동방법에서 그룹 전체를 'B'로 채우는 것과 같은 과정
            for ii in range(index, index + len(node.tokens)):
              nodes[ii] = node
            index += len(node.tokens) # 그룹 + 그 안의 총 토큰 개수만큼 이동

          case 'operator': # 연산자. nodes에 우선순위를 담아둠
            if current.code == ',': # 함수 호출의 인수들을 파싱하는 도중에 ,를 만남
              break # 일단 , 전까지 파싱함. 다음 인수는 그 위에서 파싱함
            precedence = current.precedence
            nodes[index] = precedence
            allPrecedences.add(precedence)
            index += 1

          case 'number': # 숫자. 숫자 노드를 nodes에 담아둠
            nodes[index] = Number(TokenReference.ofCount(allTokens=tokens, start=index, count=1), float(current.code))
            index += 1
          
          case _:
            index += 1
      
      limit = index
          

      # 2. Parse expression
      #   execute the operator with high precedence first sequentially
      # TODO: is there a way to optimize? This produces high complexity

      # 높은 우선순위부터 차례대로 실행
      for precedence in sorted(allPrecedences, reverse=True): # from high to low precedence
        index = start
        
        # 순차적으로 실행
        while index < limit:
          node = nodes[index]

          if type(node) == int:
            # 지금 찾는 우선순위와 같은 우선순위 발견!
            if node == precedence:
              operator = tokens[index].code

              # 왼쪽 값
              left = nodes[index - 1]
              # 오른쪽 값
              right = nodes[index + 1]

              # 위 '작동방식'에서 본 것처럼 A A A 같은 식으로 다 채워줌
              localStart = index - len(left.tokens)
              localEnd = index + len(right.tokens) + 1
              operator = BinaryOperator(
                tokens=TokenReference.ofEnd(allTokens=tokens, start=localStart, end=localEnd),
                left=left,
                right=right,
                operator=operator
              )

              for indexToSet in range(localStart, localEnd):
                nodes[indexToSet] = operator
              
              index += localEnd
            else:
              index += 1
          else:
            # 노드를 만나면
            index += 1 # 그냥 계속해서 감
      
      return nodes[start]

    
    case _:
      raise Exception(f'Malformed token: unknown token kind {firstToken.kind}')
  

In [83]:
def parseMathToAst(code: str):
  tokenizer = Tokenizer(code)
  tokens = []

  while not tokenizer.eof():
    tokens.append(tokenizer.advance())
  
  return parseToAst(tokens=tokens, nodes=[None] * len(tokens), start=0, end=len(tokens))

In [ ]:
def interpret(node: Node):
  match type(node):
    case Number:
      return node.value

    case BinaryOperator:
      left = interpret(node.left)
      right = interpret(node.right)
      operator = node.operator

      match operator:
        case '+':
          return left + right
        case '-':
          return left - right
        case '*':
          return left * right
        case '/':
          return left / right
        case '%':
          return left % right
        case '^':
          return left ** right
        case '==':
          return left == right
        case '!=':
          return left != right
        case '<':
          return left < right
        case '>':
          return left > right
        case _:
          throw Exception(f'Unknown binary operator {operator}')

In [109]:
parseMathToAst('1 + 7 * (3 - 5 * sin(123, 1))')

BinaryOperator(1.0, "+", BinaryOperator(7.0, "*", BinaryOperator(3.0, "-", BinaryOperator(5.0, "*", FunctionValue("sin", args=[123.0, 1.0])))))

In [54]:
from tkinter import Tk, Label, StringVar, Frame, Entry, Button


# Basic UI
window = Tk()
window.title('계산기')

num = StringVar()
result = StringVar()


def callback(*arg):
  s = num.get()
  # ast = parseAst(s)
  pass

resultFrame = Frame(window)

button = Button(resultFrame, text='계산', command=callback)
button.pack(side='left')

Entry(resultFrame, textvariable=num).pack(side='left')

Label(resultFrame, textvariable=result).pack(side='left')

resultFrame.pack(fill='x')

def buttonClick(key):
  num.set(num.get() + key)

# Generate buttons
lpad = Frame(window)
lpadNumbers = [
  ['7', '8', '9'],
  ['4', '5', '6'],
  ['1', '2', '3'],
  ['0', ',', '=']
]
for row in lpadNumbers:
  current = Frame(lpad)
  for column in row:
    Button(current, text=column, width=10, height=3, command=lambda *args, column=column: buttonClick(key=column)).pack(side='left')
  current.pack()

lpad.pack(side='left')


rpad = Frame(window)
rpadNumbers = [
  ['*', '/'],
  ['+', '-'],
  ['(', ')'],
  ['clear']
]

for row in rpadNumbers:
  current = Frame(rpad)
  for column in row:
    match column:
      case 'clear':
        command = lambda *args: num.set('')
      case _:
        command = lambda *args, column=column: buttonClick(key=column)
    
    Button(current, text=column, width=10, height=3, command=command).pack(side='left')
  current.pack()

rpad.pack(side='right')


window.bind('<Return>', callback)

# Blocks until window closes
window.mainloop()